# Visual SLAM Trilogy
## Part II: The Backend

In this SLAM hands-on lecture, we will implement a visual SLAM system that has an OpenCV frontend, a GTSAM backend, and a loop closure module based on a bag-of-words approach.

## Overview

The overview of our SLAM system is depicted below, simplified by certain assumptions:

1. Odometry
    - Assumes there is an odometry trajectory provided to our SLAM system.
    - In practice, this includes a Kalman filter fusing the IMU and encoder data.
2. Frontend:
    - Processes the raw sensor data and extracts relevant features for optimization.
    - Associates each measurement to a specific landmark (3D point).
    - Provide initial values for the backend variables.
3. Mapping
    - Utilizes a very minimum sparse map.
    - Could be replaced with OGM or even 3D Gaussian Splatting in the future.
4. Backend
    - Solve the maximum a posteriori (MAP) estimation problem.
    - Feed back information to loop closure.
5. Loop closure:
    - Acts as a long-term tracking module (compared to the short-term tracking module in frontend).
    - Implemented with visual bag-of-word algorithm.

![slam_overview](assets/slam_overview.png)

## Dataset

We will use the abandoned_factory P006 sequence from the TartanAir dataset to test the system. It is a simulation dataset with diverse environments and ground truth dataset, which make it perfect for testing and evaluating our system. To get started, we'll need to access the camera intrinsics, extrinsics, and data format information, which can be found here: https://github.com/castacks/tartanair_tools/blob/master/data_type.md.

## Implementation

In this notebook, we will walk through the implementation of the camera model, backend, and loop closure step-by-step, while visualizing the output of each step. Specifically, we will cover the following topics:

- Calculating global XYZ positions for the landmarks
- Optimize the trajectory with projection factors in GTSAM
- Improve estimated trajectory by adding loop closure

## I. Dependency


### 1. Install Python libraries

In [ ]:
# install the minslam package in “editable” mode
# !pip install -e ..

# install other libraries
# !pip install numpy spatialmath-python opencv-python matplotlib gtsam ipympl evo plotly

### 2. Import libraries and dataset
Please download [abadoned_factory P006 dataset](https://drive.google.com/file/d/1Q_fSI0U-IMfv90lyE1Uh78KV2QJheHbv/view?usp=share_link) and extract it to a folder named "data".

In [ ]:
# this block should run without error

# dataset
import os

# change the perspective of 3d plots with this command
%matplotlib widget

# test if we can find the dataset
dataset_folder = '../data/P006/'
print('Check if the data folder exists:',os.path.exists(dataset_folder))

# visualization
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# frontend
import numpy as np
from spatialmath import *

# backend
import gtsam
from gtsam.symbol_shorthand import L, X

# our slam implementation
from minslam.data_loader import TartanAirLoader, plot_trajectory
from minslam.frontend import Frontend
from minslam.params import Params
from minslam.backend import Backend
from minslam.camera import PinholeCamera

### 3. Load images and trajectory

In [ ]:
traj_filename = 'pose_left.txt'
traj_path = os.path.join(dataset_folder, traj_filename)
print('Loading trajectory from ', traj_path)

# load a trajectory
dataset = TartanAirLoader('../data/P006')
dataset.load_ground_truth()
gt_poses = dataset.gt
start_index = 250
odom_poses = dataset.add_noise(gt_poses, [3e-4, 3e-4], [1e-3, 3e-4], seed=100, start=start_index)
dataset.set_odometry(odom_poses)

## II. Frontend

We will build a function that tries to find the next keyframe and return it.

In [ ]:
params = Params('../params/tartanair.yaml')
frontend = Frontend(params)

def run_frontend_once(frontend):
    pose = dataset.read_current_odometry()
    while not frontend.keyframe_selection(pose):
        if not dataset.load_next_frame():
            break
        pose = dataset.read_current_odometry()
    color, depth = dataset.read_current_rgbd()
    frontend.add_keyframe(pose, color, depth, dataset.curr_index)
    print(f'--- Added keyframe {frontend.frame_id} (seq id: {dataset.curr_index}) ---')
    more_points_n = params['frontend']['feature']['number']
    frontend.extract_features(more_points_n, append_mode=False)
    print('extracting features:', len(frontend.curr_frame.keypoints), f'(expected {more_points_n})')
    if frontend.frame_id > 0:
        frontend.match_features()
        print('matching features:', len(frontend.curr_frame.matches))
        frontend.eliminate_outliers()
    frontend.assign_global_id()
    # frontend.plot_matches(with_global_id=True)
    return frontend.curr_frame

## III. Camera

In this section, we will learn what is pinhole camera model and how to use it to calculate the XYZ global position of the landmarks.

### 1. Pinhole camera model

To help us understand the pinhole camera model, we recommend reviewing two resources:

- [CS231A Course Notes 1: Camera Models](https://web.stanford.edu/class/cs231a/course_notes/01-camera-models.pdf): how to derive the pinhole camera model.
- [OpenCV - Camera Calibration and 3D Reconstruction](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html): how a OpenCV library models and implements pinhole camera.

These resources should provide a good foundation for understanding the principles behind the pinhole camera model and how it is implemented in practice.

In [ ]:
camera = PinholeCamera(params)

### 2. Calculate the global XYZ positions of landmarks

Next, we will use the camera intrinsics and the pixel coordinates of each feature to calculate its 3D position in the camera frame. We will then use the camera poses and the 3D positions of the features to calculate the global XYZ positions of the landmarks.

In [ ]:
dataset.set_curr_index(100)
frontend_frame = run_frontend_once(frontend)
color = frontend_frame.color
depth = frontend_frame.depth


fig_color = px.imshow(color[:,:,::-1])
fig_color.update_traces(hoverinfo="x+y+z", name="")
fig_color.show()

clipped_depth = depth.clip(0, 40)
fig_depth = px.imshow(clipped_depth, color_continuous_scale='gray')
fig_depth.update_traces(hoverinfo="x+y+z", name="")
fig_depth.show()

To calculate the global positions, we will first use the camera model to get the XYZ position of the landmarks in the camera frame (x->right, y->downward, z->forward). The point cloud below is generated by evenly sampled points from the depth image.

In [ ]:
sample_stride = 1
sample_number = int(640*480/sample_stride/sample_stride)
points_xyz = np.zeros((sample_number+1, 3)) # add one row for the origin
points_color = np.zeros((sample_number+1, 3))
index = 0
for i in range(0,640,sample_stride):
    for j in range(0,480,sample_stride):
        depth = frontend_frame.depth[j, i]
        # skip points with large depth for better visualization
        if depth>50:
            continue
        ############## transform these points from 2d to 3d ##############
        points_xyz[index] = camera.back_project2(i, j, depth).flatten()
        ##################################################################
        points_color[index] = frontend_frame.color[j, i][::-1]
        index += 1
points_xyz = points_xyz[:index+1]

fig = go.Figure(data=[go.Scatter3d(
    x=points_xyz[:,0],
    y=points_xyz[:,2],
    z=points_xyz[:,1],
    mode='markers',
    marker=dict(
        size=2,
        color=points_color,
    )
)])

# Set the aspect ratio
fig.update_layout(
    width=1000,
    height=800,
    scene=dict(
        aspectmode='data',
        zaxis=dict(autorange="reversed")
    ),
    scene_xaxis_title='X',
    scene_yaxis_title='Z',
    scene_zaxis_title='Y'
)

fig.show()

In real backend, we will only use the detected sparse feature, and they look like this:

In [ ]:
dataset.set_curr_index(100)
frontend_frame = run_frontend_once(frontend)
points_xyz = np.zeros((len(frontend_frame.points)+1, 3))
points_color = np.zeros((len(frontend_frame.points)+1, 3))
index = 0
for point in frontend_frame.points:
    i,j = [int(x) for x in point]
    depth = frontend_frame.depth[j, i]
    # skip points with large depth for better visualization
    if depth>50:
        continue
    ############## transform these points from 2d to 3d ##############
    points_xyz[index] = camera.back_project2(i, j, depth).flatten()
    ##################################################################
    points_color[index] = frontend_frame.color[j, i][::-1]
    index += 1

fig = go.Figure(data=[go.Scatter3d(
    x=points_xyz[:,0],
    y=points_xyz[:,2],
    z=points_xyz[:,1],
    mode='markers',
    marker=dict(
        size=2,
        color=points_color,
    )
)])

# Set the aspect ratio
fig.update_layout(
    width=1000,
    height=800,
    scene=dict(
        aspectmode='data',
        zaxis=dict(autorange="reversed")
    ),
    scene_xaxis_title='X',
    scene_yaxis_title='Z',
    scene_zaxis_title='Y'
)

fig.show()

As we can see that the features are sparse, it is not suitable for visualization. So, we will continue to use sampled points in next step.

In this step, we will transform these points from camera frame to the world frame using the odometry pose.

In [ ]:
sample_stride = 1
sample_number = int(640*480/sample_stride/sample_stride)
points_xyz = np.zeros((sample_number+1, 3)) # add one row for the origin
points_color = np.zeros((sample_number+1, 3))
index = 0
for i in range(0,640,sample_stride):
    for j in range(0,480,sample_stride):
        depth = frontend_frame.depth[j, i]
        # skip points with large depth for better visualization
        if depth>50:
            continue
        ############# transform these points from 2d to 3d ############
        points_xyz[index] = camera.back_project(i, j, depth, frontend_frame.odom_pose).flatten()
        ################################################################
        points_color[index] = frontend_frame.color[j, i][::-1]
        index += 1
points_xyz = points_xyz[:index+1]

fig = go.Figure(data=[go.Scatter3d(
    x=points_xyz[:,2],
    y=points_xyz[:,0],
    z=points_xyz[:,1],
    mode='markers',
    marker=dict(
        size=2,
        color=points_color,
    )
)])

# Set the aspect ratio
fig.update_layout(
    width=1000,
    height=800,
    scene=dict(
        aspectmode='data',
        zaxis=dict(autorange="reversed")
    ),
    scene_xaxis_title='Z',
    scene_yaxis_title='X',
    scene_zaxis_title='Y'
)


This looks very familiar to the previous plot... So why do we want to transform it into the world frame? First, it enables us to provide an initial guess for the landmarks in backend. Secondly, we can simply stack these point cloud together to check the odometry poses. For example, run this block to see what happens if we have a bad or even wrong odometry:

In [ ]:
dataset = TartanAirLoader('../data/P006/')
dataset.load_ground_truth()
gt_poses = dataset.gt

# uncomment this part to use gt
dataset.set_odometry(gt_poses)

# uncomment this part to add small noise
# odom_poses = dataset.add_noise(gt_poses, [1e-3, 3e-3], [1e-3, 1e-3], seed=100, start=start_index)
# dataset.set_odometry(odom_poses)

# uncomment this part to use wrong odometry
# T = SE3([
#     [1, 0, 0, 0],
#     [0, 0, 1, 0],
#     [0, 1, 0, 0],
#     [0, 0, 0, 1]
# ])
# odom_poses = []
# for i in range(1, len(gt_poses)):
#     odom_poses += [T*gt_poses[0]]
# dataset.set_odometry(odom_poses)

def get_world_points(frame_id, sample_stride=5):
    dataset.set_curr_index(frame_id)
    frontend_frame = run_frontend_once(frontend)
    color = frontend_frame.color
    depth = frontend_frame.depth
    sample_number = int(640*480/sample_stride/sample_stride)
    points_xyz = np.zeros((sample_number+1, 3))
    points_color = np.zeros((sample_number+1, 3))[::-1]
    index = 0
    for i in range(0,640,sample_stride):
        for j in range(0,480,sample_stride):
            depth = frontend_frame.depth[j, i]
            if depth>50:
                continue
            points_xyz[index] = camera.back_project(i, j, depth, dataset.read_current_odometry()).flatten()
            points_color[index] = frontend_frame.color[j, i]
            index += 1
    return points_xyz[:index+1], points_color[:index+1]

all_points = [get_world_points(i) for i in range(100, 300, 10)]
stacked_point_cloud = np.vstack([x[0] for x in all_points])
stacked_point_cloud_color = np.vstack([x[1] for x in all_points])

fig = go.Figure(data=[go.Scatter3d(
    x=stacked_point_cloud[:,2],
    y=stacked_point_cloud[:,0],
    z=stacked_point_cloud[:,1],
    mode='markers',
    marker=dict(
        size=2,
        color=stacked_point_cloud_color,
    )
)])


# Set the aspect ratio
fig.update_layout(
    width=1000,
    height=800,
    scene=dict(
        aspectmode='data',
        zaxis=dict(autorange="reversed")
    ),
    scene_xaxis_title='Z',
    scene_yaxis_title='X',
    scene_zaxis_title='Y'
)

fig.show()

## IV. Backend

In this section, we will use GTSAM library to optimize the trajectory using the measurements obtained from the frontend. To incorporate the camera pose and landmark positions in the optimization process, we will use the projection factor in GTSAM.

In [ ]:
# load the dataset
params = Params('../params/tartanair.yaml')
dataset = TartanAirLoader('../data/P006/')
frontend = Frontend(params)
backend = Backend(params)
dataset.set_curr_index(start_index)
n_keyframes = 200

# read the ground truth and odometry
dataset.load_ground_truth()
gt_poses = dataset.gt
odom_poses = dataset.add_noise(gt_poses, [1e-4, 3e-5], [1e-3, 3e-4], seed=100, start=start_index)
dataset.set_odometry(odom_poses)

# for plotting
gt_traj = np.zeros((n_keyframes, 3))
odom_traj = np.zeros((n_keyframes, 3))

# run the frontend and backend
for i in range(n_keyframes):
    # get results from frontend
    frontend_frame = run_frontend_once(frontend)
    gt_traj[i] = dataset.read_current_ground_truth().t
    odom_traj[i] = dataset.read_current_odometry().t
    measurements = []
    frame_id = frontend_frame.frame_id
    count = 0
    for landmark in frontend_frame.landmarks:
        global_id = landmark.global_id
        measurement = landmark.measurements[frame_id] # u, v, depth
        measurements.append((global_id, *measurement))
        count += 1
    print(f'add {count} measurements to backend')
    # add measurements to backend
    backend.add_keyframe(frame_id, frontend_frame.odom_pose, measurements)
# optimize the backend
backend.optimize(optimizer='LM')
backend_estimate = backend.current_estimate
estimated_traj = gtsam.utilities.extractPose3(backend_estimate)[:, -3:]

# plot the results
fig = go.Figure()
plot_trajectory(odom_traj[:n_keyframes], 'odom', fig)
plot_trajectory(gt_traj[:n_keyframes], 'gt', fig)
plot_trajectory(estimated_traj[:n_keyframes], 'estimated', fig)

In [ ]:
# load the dataset
dataset.set_curr_index(start_index)

# create frontend and backend
params = Params('../params/tartanair.yaml')
frontend = Frontend(params)
backend = Backend(params)
n_keyframes = 100

# for plotting
gt_traj = np.zeros((n_keyframes, 3))
odom_traj = np.zeros((n_keyframes, 3))

# parameters for the backend
initial_estimate = gtsam.Values()
graph = gtsam.NonlinearFactorGraph()
pose_prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.03]*3+[0.01]*3))
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.3)
noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 1)
noise_model_robust = gtsam.noiseModel.Robust.Create(gtsam.noiseModel.mEstimator.Huber.Create(1.345), noise_model)
fx, fy, cx, cy = camera.camera_matrix
K = gtsam.Cal3_S2(fx, fy, 0.0, cx, cy)

total_measurements = 0

# run the frontend and backend
for i in range(n_keyframes):
    # get results from frontend
    frontend_frame = run_frontend_once(frontend)
    gt_traj[i] = dataset.read_current_ground_truth().t
    odom_traj[i] = dataset.read_current_odometry().t
    measurements = []
    frame_id = frontend_frame.frame_id
    for landmark in frontend_frame.landmarks:
        global_id = landmark.global_id
        measurement = landmark.measurements[frame_id] # u, v, depth
        measurements.append((global_id, *measurement))

    # add measurements to backend
    odom_pose = frontend_frame.odom_pose
    pose = gtsam.Pose3(gtsam.Rot3(odom_pose.R), gtsam.Point3(odom_pose.t))
    # add initial estimate
    initial_estimate.insert(X(frame_id), pose)
    if i == 0:
        # add prior for first frame
        graph.add(gtsam.PriorFactorPose3(X(frame_id), pose, pose_prior_noise))
    count = 0
    for global_id, u, v, depth in measurements:
        # add measurements
        global_pos = camera.back_project(u, v, depth, odom_pose).flatten()
        if initial_estimate.exists(L(global_id)):
            init_pos = initial_estimate.atVector(L(global_id))
            # calc error
            error = np.linalg.norm(init_pos - global_pos)
            if error>0.5:
                continue
        graph.add(gtsam.GenericProjectionFactorCal3_S2(
            np.array([u, v]),
            noise_model_robust,
            X(frame_id),
            L(global_id),
            K
        ))
        count += 1
        if total_measurements<50:
            total_measurements += 1
            graph.add(gtsam.PriorFactorPoint3(L(global_id), global_pos, point_noise))
        if not initial_estimate.exists(L(global_id)):
            initial_estimate.insert(L(global_id), global_pos)
    print(f'add {count} measurements to backend')
# optimize
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate)
current_estimate = optimizer.optimize()
estimated_traj = gtsam.utilities.extractPose3(current_estimate)[:, -3:]


# plot the results
fig = go.Figure()
plot_trajectory(odom_traj[:n_keyframes], 'odom', fig)
plot_trajectory(gt_traj[:n_keyframes], 'gt', fig)
plot_trajectory(estimated_traj[:n_keyframes], 'estimated', fig)